In [1]:
!pip uninstall kaggle -y
!pip uninstall --upgrade.pip
!pip install kaggle==1.5.6
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

Found existing installation: kaggle 1.5.12
Uninstalling kaggle-1.5.12:
  Successfully uninstalled kaggle-1.5.12

Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: --upgrade.pip
     |████████████████████████████████| 58 kB 2.8 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72857 sha256=555ca35ed11d4c7d50785f33552833cd76423ad80e7982f74cb74b29e686795b
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle


In [ ]:
!kaggle competitions download -c 2021-ai-w4-p2

  0% 0.00/67.9k [00:00<?, ?B/s]
100% 67.9k/67.9k [00:00<00:00, 41.8MB/s]


In [ ]:
!ls

2021-ai-w4-p2.zip  drive  sample_data


In [ ]:
!unzip 2021-ai-w4-p2.zip

Archive:  2021-ai-w4-p2.zip
  inflating: sample.csv              
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
submit=pd.read_csv("sample.csv")

In [ ]:
import torch
import torch.optim as optim

torch.manual_seed(1)

In [ ]:
train

,N,P,K,temperature,humidity,ph,rainfall,label
0,22,36,16,30.581395,50.771481,8.184229,64.585596,mothbeans
1,18,27,41,22.365094,92.308824,7.175344,104.821633,pomegranate
2,61,68,50,35.214628,91.497251,6.793245,243.074507,papaya
3,26,72,22,28.767949,37.577921,4.674942,91.720849,pigeonpeas
4,2,140,197,22.697801,92.822234,5.534567,105.050823,apple
...,...,...,...,...,...,...,...,...
1645,10,5,5,21.213070,91.353492,7.817846,112.983436,orange
1646,108,94,47,27.359116,84.546250,6.387431,90.812505,banana
1647,11,36,31,27.920633,51.779659,6.475449,100.258567,mango
1648,11,124,204,13.429886,80.066340,6.361141,71.400430,grapes


In [ ]:
x=train.drop(["label"],axis=1)
y=train["label"]

In [ ]:
y

0         mothbeans
1       pomegranate
2            papaya
3        pigeonpeas
4             apple
           ...     
1645         orange
1646         banana
1647          mango
1648         grapes
1649         lentil
Name: label, Length: 1650, dtype: object

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
x_std=scaler.transform(x)
test_std=scaler.transform(test)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_re=le.fit(y)
y=le.transform(y)

In [ ]:
import numpy as np
x=np.array(x_std)
x=torch.FloatTensor(x)
y=torch.LongTensor(y)
test=np.array(test_std)
test=torch.FloatTensor(test)

In [ ]:
x.shape[1]

7

In [ ]:
len(y.unique())

22

In [ ]:
linear1 = torch.nn.Linear(7, 256, bias=True)
linear2 = torch.nn.Linear(256, 512, bias=True)
linear3 = torch.nn.Linear(512, 256, bias=True)
linear4 = torch.nn.Linear(256, 22, bias=True)
relu=torch.nn.ReLU()
dropout=torch.nn.Dropout(0.1)

In [ ]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)

Parameter containing:
tensor([[ 0.0242,  0.0256, -0.0319,  ..., -0.1302,  0.0046, -0.1331],
        [-0.1342, -0.0329,  0.0107,  ...,  0.1364,  0.0661, -0.0090],
        [ 0.0153,  0.1253,  0.1081,  ..., -0.1425, -0.0973, -0.0027],
        ...,
        [ 0.0803, -0.0410,  0.0715,  ..., -0.0563,  0.0096, -0.0200],
        [ 0.0642,  0.1249,  0.0693,  ..., -0.0353, -0.0414, -0.0027],
        [ 0.0783,  0.0982, -0.1394,  ..., -0.0700,  0.0701, -0.0375]],
       requires_grad=True)

In [ ]:
model = torch.nn.Sequential(linear1,relu,dropout, linear2,relu,dropout,linear3,relu,dropout,linear4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss = torch.nn.CrossEntropyLoss()

In [ ]:
for stop in range(250):

    optimizer.zero_grad()

    hypothesis=model(x)

    cost=loss(hypothesis,y)#input=(N,C),target=(N)

    cost.backward()

    optimizer.step()
    if stop%20==0:
      print(stop, cost.item())

0 3.1007723808288574
20 0.3620319962501526
40 0.14917346835136414
60 0.07182899862527847
80 0.07171938568353653
100 0.0396617129445076
120 0.03416160121560097
140 0.02630797028541565
160 0.02783694863319397
180 0.022132329642772675
200 0.025404155254364014
220 0.02555234543979168
240 0.01802254281938076


In [ ]:
prediction=list()
with torch.no_grad():
    pred = model(test)
    _,pred =torch.max(pred,1)
    prediction.extend(pred.tolist())
    prediction = y_re.inverse_transform(prediction)

In [ ]:
submit['label'] = prediction

In [ ]:
submit

,id,label
0,0,muskmelon
1,1,watermelon
2,2,papaya
3,3,papaya
4,4,apple
...,...,...
545,545,coffee
546,546,papaya
547,547,chickpea
548,548,coffee


In [ ]:
submit.to_csv("submit.csv",mode='w',index=False)

In [ ]:
!kaggle competitions submit -c 2021-ai-w4-p2 -f submit.csv -m "Message"

100% 6.46k/6.46k [00:00<00:00, 7.52kB/s]
Successfully submitted to 재배환경 별 작물 종류 예측